In [ ]:
import numpy as np
from torchvision.datasets import CIFAR10
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import transforms
import torch, torch.nn as nn
import torch.nn.functional as F

from torchvision.models import efficientnet_v2_s, resnet18

In [ ]:
means = np.array((0.4914, 0.4822, 0.4465))
stds = np.array((0.2023, 0.1994, 0.2010))

transform_augment = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
#     transforms.Resize((32, 32)),
    transforms.RandomRotation([-30, 30]),
    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
    transforms.ColorJitter(brightness=0.02, contrast=0.001, saturation=0.2), # Set the color params
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(means, stds),
])


transform_augment_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(means, stds)])

batch_size = 400


train_loader = CIFAR10("./cifar_data/", train=True, transform=transform_augment)
trainloader = torch.utils.data.DataLoader(train_loader, 
                                          batch_size=batch_size,
                                          shuffle=True, 
                                          num_workers=2)


test_loader = CIFAR10("./cifar_data/", train=False, transform=transform_augment_test)
testloader = torch.utils.data.DataLoader(test_loader, 
                                         batch_size=batch_size,
                                         shuffle=False, 
                                         num_workers=2)

In [ ]:
import matplotlib.pyplot as plt
examples = next(iter(trainloader))

img = examples[0]


In [ ]:
plt.imshow(np.transpose(img[0].numpy()))

In [ ]:
import sys
 
# setting path
sys.path.append('../')
from models import train_model

In [ ]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4), 
                                        nn.Flatten(), 
                                        nn.Linear(512, num_classes))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [ ]:
num_classes = len(train_loader.classes)
num_classes

In [ ]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1, weight = None):
        """if smoothing == 0, it's one-hot method
           if 0 < smoothing < 1, it's smooth method
        """
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.weight = weight
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        assert 0 <= self.smoothing < 1
        pred = pred.log_softmax(dim=self.dim)

        if self.weight is not None:
            pred = pred * self.weight.unsqueeze(0)

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [ ]:
device = ("cuda" if torch.cuda.is_available() else 'cpu')
# model = ResNet9(3, 10).to(device)

model = resnet18(weights='DEFAULT')
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

model = model.to(device)



weight_decay = 1e-4

optimizer = torch.optim.Adam(model.parameters(), lr= 0.001) # 0.001 !
# optimizer = torch.optim.Adam(model.parameters(), lr= 0.001, weight_decay=weight_decay) # 0.001 !
# criterion = torch.nn.CrossEntropyLoss()
criterion = LabelSmoothingCrossEntropy(classes=num_classes, smoothing=0.3, dim=-1, weight = None)

tb = SummaryWriter()
model, optimizer = train_model(model, criterion, 
                    optimizer, train_dataloader=trainloader, test_dataloader=testloader, 
                    device=device,
                    n_epochs=100, batch_size=batch_size)

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of test images: {100 * correct // total} %')

In [ ]:
checkpoint = {
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()}
torch.save(checkpoint, 'resnet.pth')